In [6]:
%pip install datasets trl peft bitsandbytes wandb accelerate transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install unsloth
%pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Note: you may need to restart the kernel to use updated packages.
Found existing installation: unsloth 2024.9.post2
Uninstalling unsloth-2024.9.post2:
  Successfully uninstalled unsloth-2024.9.post2
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-cthm5x3i/unsloth_db8ab60368014c7295e9c3e9b2370de7
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-cthm5x3i/unsloth_db8ab60368014c7295e9c3e9b2370de7
  Resolved https://github.com/unslothai/unsloth.git to commit fb77505f8429566f5d21d6ea5318c342e8a67991
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.9.post2-py3-none-any.whl size=154625 sha256=644fe0cd421067987dda33a12af2adc7d379a7d2f90a02aec3ea0db488950d8b
  Stored in directory: /tmp/pip-ephem-wheel-cache-bi47vhky/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d83

In [3]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [4]:
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [5]:
from hmac import new
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
base_model = "unsloth/llama-3-8b-Instruct-bnb-4bit"
new_model = "./output/llama-3-8b-Instruct-bnb-4bit"
final_ckpt = "./output/final_ckpt"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model, # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla V100-SXM2-32GB. Max memory: 31.739 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2024.9.post2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [9]:
from datasets import load_dataset

dataset = load_dataset("Intel/orca_dpo_pairs")['train']

In [10]:
dataset = dataset.shuffle(seed=42).select(range(150))

In [11]:
dataset

Dataset({
    features: ['system', 'question', 'chosen', 'rejected'],
    num_rows: 150
})

In [12]:
dataset[47]['question']

'Article: People like to read stories about dog very much.They thought dogs were very clever. One of my good friend , Bob , had a very big dog .Its name was Bill.Every Sunday afternoon,Bob and Bill had a walk in the park , Bill liked walking with Bob very much. I visited Bob last Sunday.I stayed in his house for a long time.Bob and I talked with each other happily.Soon it was time for them to go for a walk in the park .We forgot that.Bill began to worry about it.He walked around the room and sat down in front of me and looked at me.But I didn\'t know this.I went on talking with my friend.At last, Bill couldn\'t wait.He went out of the room and came back soon.He carried my hat in his mouth.Oh, I knew what Bill meant. Question: How many people were there in this story? Yes or no, is the answer "Two"?\nA:'

In [13]:
dataset[47]['rejected']

' Sure, I\'d be happy to help! To answer your question, the story has three people:\n\n1. Bob (the owner of the dog)\n2. Bill (the dog)\n3. You (the person who visited Bob and is asking the question)\n\nSo, the answer to your question is "No" because there are not two people in the story. There are three people.'

In [14]:
dataset[47]['chosen']

'In this story, there were three people mentioned: the narrator, their good friend Bob, and the person visiting Bob on Sunday. So the answer to the question "How many people were there in this story?" is three. Therefore, "Two" is not the correct answer, so the response for "Yes or no, is the answer \'Two\'?" would be no.'

In [15]:
chat_template = open('llama-3-instruct.jinja').read()
chat_template

"{% if messages[0]['role'] == 'system' %}\n    {% set offset = 1 %}\n{% else %}\n    {% set offset = 0 %}\n{% endif %}\n\n{{ bos_token }}\n{% for message in messages %}\n    {% if (message['role'] == 'user') != (loop.index0 % 2 == offset) %}\n        {{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}\n    {% endif %}\n\n    {{ '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n' + message['content'] | trim + '<|eot_id|>' }}\n{% endfor %}\n\n{% if add_generation_prompt %}\n    {{ '<|start_header_id|>' + 'assistant' + '<|end_header_id|>\\n\\n' }}\n{% endif %}"

In [16]:
chat_template = chat_template.replace('    ', '').replace('\n', '')
tokenizer.chat_template = chat_template
chat_template

"{% if messages[0]['role'] == 'system' %}{% set offset = 1 %}{% else %}{% set offset = 0 %}{% endif %}{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == offset) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{{ '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n' + message['content'] | trim + '<|eot_id|>' }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>' + 'assistant' + '<|end_header_id|>\\n\\n' }}{% endif %}"

In [17]:
def dataset_format(example):
    # Format system
    if len(example["system"]) > 0:
        message = {"role": "system", "content": example["system"]}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""
    # Format instruction
    message = {"role": "user", "content": example["question"]}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    # Format chosen answer
    chosen = example["chosen"] + "<|eot_id|>\n"
    # Format rejected answer
    rejected = example["rejected"] + "<|eot_id|>\n"
    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

In [18]:
original_columns = dataset.column_names
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

dataset = dataset.map(
    dataset_format,
    remove_columns=original_columns,
    num_proc=os.cpu_count(),
)

In [19]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: murphypei. Use `wandb login --relogin` to force relogin


True

In [20]:
from trl import DPOConfig, DPOTrainer
from unsloth import is_bfloat16_supported


dpo_config = DPOConfig(
    output_dir=new_model,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=50,  # tweak this to change # of steps in the training run
    save_strategy="no",
    logging_steps=1,
    optim="paged_adamw_32bit",
    warmup_steps=10,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    report_to="wandb",
    beta=0.1,
    max_prompt_length=512,
    max_length=1024,
    force_use_ref_model=True,
    remove_unused_columns=False,
)

In [21]:
dpo_trainer = DPOTrainer(
    model,
    None,
    args=dpo_config,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

Tokenizing train dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

Detected kernel version 4.9.70, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [22]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 150 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.693100,0.000000,0.000000,0.000000,0.000000,-190.893845,-210.536240,-0.043843,0.171945
2,0.693100,0.000000,0.000000,0.000000,0.000000,-245.101746,-325.451477,-0.079173,0.098809
3,0.691400,-0.001516,-0.005003,0.625000,0.003486,-217.521164,-179.730484,0.079622,0.499396
4,0.683300,0.005289,-0.014467,0.875000,0.019756,-210.976364,-219.451324,0.001635,0.290597
5,0.672400,0.010789,-0.031176,1.000000,0.041965,-192.239517,-153.110504,0.097465,0.438388
6,0.641000,0.048883,-0.058598,1.000000,0.107481,-210.323975,-194.566971,0.180635,0.628023
7,0.565600,0.134390,-0.141556,1.000000,0.275945,-231.015640,-325.815735,0.220533,0.046113
8,0.525000,0.136156,-0.238817,1.000000,0.374973,-287.503296,-350.799194,0.012893,0.112961
9,0.439500,0.219233,-0.401977,1.000000,0.621210,-238.072388,-256.310059,-0.044768,0.021856
10,0.428300,0.298181,-0.348656,1.000000,0.646837,-165.312408,-132.984741,-0.028148,0.176269


TrainOutput(global_step=50, training_loss=0.15873264772686524, metrics={'train_runtime': 478.5969, 'train_samples_per_second': 0.836, 'train_steps_per_second': 0.104, 'total_flos': 0.0, 'train_loss': 0.15873264772686524, 'epoch': 2.6666666666666665})

In [26]:
# dpo_trainer.model.save_pretrained("output/final_ckpt")
# tokenizer.save_pretrained("output/final_ckpt")
model.save_pretrained_merged(final_ckpt, tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 148.33 out of 251.58 RAM for saving.


100%|██████████| 32/32 [00:01<00:00, 31.65it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [28]:
import gc

# Flush memory
del dpo_trainer, model
gc.collect()
torch.cuda.empty_cache()

NameError: name 'dpo_trainer' is not defined

In [27]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = final_ckpt, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla V100-SXM2-32GB. Max memory: 31.739 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

output/final_ckpt does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(new_model)
tokenizer

PreTrainedTokenizerFast(name_or_path='./output/llama-3-8b-Instruct-bnb-4bit', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>', 'pad_token': '<|reserved_special_token_250|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|reserved_special_token_2|>", rstrip=False, lstrip=False, single_word=False, norma

In [31]:
# Format prompt
messages = [
    {"role": "system", "content": "You are a helpful assistant chatbot that provides concise answers."},
    {"role": "user", "content": "What are GPUs and why would I use them for machine learning tasks?"},
]

inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
inputs

tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,  11190,
          18328,   6369,   6465,    430,   5825,  64694,  11503,     13, 128009,
         128006,    882, 128007,    271,   3923,    527,  71503,    323,   3249,
           1053,    358,   1005,   1124,    369,   5780,   6975,   9256,     30,
         128009, 128006,  78191, 128007,    271]], device='cuda:0')

In [32]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant chatbot that provides concise answers.<|eot_id|><|start_header_id|>user<|end_header_id|>

What are GPUs and why would I use them for machine learning tasks?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A GPU (Graphics Processing Unit) is a specialized electronic circuit designed to quickly manipulate and alter memory to accelerate the creation of images on a computer screen. However, in the context of machine learning, GPUs are used to significantly speed up the training and inference processes.

GPUs are particularly useful for machine learning tasks because they:

1. **Handle massive parallel processing**: GPUs have thousands of cores, which can perform many calculations simultaneously, making them much faster than traditional CPUs (Central Processing Units) for tasks that require heavy matrix multiplication, such as neural network training.
2. **Provide high memory bandwidth**: GPUs h